<a href="https://colab.research.google.com/github/RamSankarTheDeveloper/Question-generation-algorithm-based-on-blooms-taxonomy/blob/preprocessing/preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#note:restart the kernel if you had already imported another spacy version in collab
#date:21/02/2023
#-----------------------------------------
#https://youtu.be/Sp6jZFHI02Y
!git clone https://github.com/huggingface/neuralcoref.git 
import os
os.chdir("/content/neuralcoref")
!pip install -r requirements.txt
!pip install -e .  #'.' is reqd
#---------------------------------------
!pip install spacy==2.3.0 
import spacy #after restarting the kernel
print(spacy. __version__)
#-----------------------------------------
import spacy.cli 
spacy.cli.download("en")
#so that "en" works
#OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
#https://stackoverflow.com/questions/62822737/oserror-e050-cant-find-model-de-it-doesnt-seem-to-be-a-shortcut-link-a
#-------------------------------------------
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')

#Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.

In [ ]:
import pandas as pd
import numpy as np
#import spacy
nlp1 = spacy.load("en_core_web_sm")
import nltk
nltk.download('all')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import PorterStemmer
pst = PorterStemmer()
import re


In [73]:
class preprocess:

  def convert_para_to_listofsents(self, text):
    doc=nlp1(text)
    token_sentences=[]
    string_sentences=[]
    for sent in doc.sents:
      token_sentences.append(sent)
      string_sentences.append(sent.text)
    return  string_sentences #,token_sentences,

  def convert_to_noun_phrases(self,sentence):
    doc =nlp1(sentence)
    a=[]
    for np in doc.noun_chunks:
      a.append(np.text)
    return a

  def convert_to_tokens(self,sentence):
    #nltk
    token_list = word_tokenize(sentence)
    #for i in range(len(token_list)):
     # token_list[i]=token_list[i].text
    return token_list

  def convert_to_stopwords(self,sentence):
    sp=spacy.load("en_core_web_sm")
    all_stopwords = sp.Defaults.stop_words
    text_tokens = word_tokenize(sentence)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]
    for i in tokens_without_sw:
      tokens_without_sw[i]=tokens_without_sw[i].text
    return tokens_without_sw

  def convert_to_basewords(self,sentence):
    a=[]
    doc=nlp1(sentence)
    for token in doc:
      a.append(token.lemma_)
    for i in range(len(a)):
      a[i]=pst.stem(a[i])
    return a
    
  def convert_listitems_to_lowercase(self,list1):
    for i in range(len(list1)):
      list1[i]=list1[i].cnvrt_to_lwr()
    return list1

  def replace_all_corefs(self,text,k=0): 
    #change it into different versions
    #https://stackoverflow.com/questions/6570635/installing-multiple-versions-of-a-package-with-pip
    doc = nlp(text)
    edited_text = doc._.coref_resolved
    nouns = doc._.coref_clusters
    items=nouns
    for i in range(len(nouns)):
      nouns[i]=str(nouns[i]).split(":")[0]
    if k == 0:
      return edited_text
    else :
      return edited_text , items

  def convert_to_ngrams(self,sentence, n):
    manygrams = list(ngrams(sentence.split(), n))
    for i in manygrams:
      manygrams[i]=manygrams[i].text
    return manygrams
  


In [17]:
############################################level_1###############################################

In [38]:
class l1:

  def __init__ (self, paragraph= "John is stronger than Johny"):
    self.paragraph = paragraph
    self.coref_text = preprocess().replace_all_corefs(paragraph)
    self.dep()
    self.ent()


  def dep(self):
    doc = nlp1(self.coref_text)
    subj=root_verb=obj =[]
    for token in doc:
      if token.dep_ in ["nsubjpass",'csubjpass','nsubj','csubj']:
        subj.append(token)
      if token.dep_ in ["ROOT","root"]:
        root_verb.append(token)
      if token.dep_ in ["dobj",'iobj','oprd','obj',"pobj"]:
        obj.append(token)
      return subj,root_verb, obj
      
  def ent(self):
    ent_list=[]
    for ent in doc.ents:
      ent_list.append(ent)
    return ent_list

#ent_label_list.append(entities.label_)

"""if item in entity list is present in subj/verb/obj,
          then assign appropriate wh-question"""

 def iterate_sentence(self):
   sentence_list = preprocess().convert_para_to_listofsents(self.coref_text)
   for sentence in sentence_list:
     sentence = preprocess().convert_to_tokens(sentence)
     if any(x in sentence for x in self.ent_list): #checks if sentence contains entities
     self.question(sentence)
       

 def question(self, sentence):
   for i in range(len(sentence)):
      for ent in self.ent_list:
        if sentence[i] == ent.label:
          print(ent.label)




     
if any(x in list1 for x in list2):
    print("Duplicates found.")
else:
    print("No duplicates found.")
      


check_ent_dep()
def question(wh,name):
  question=[]
  for item1 in sent_list:
    #dict/df/list/class attributes/df+ML_model
    if item1.text==name.text:
    #check multiple words
      question.append(wh)
    else:
      question.append(item1.text)
  print("\n",*question,"?",end=" ")
for item in ent_text_list:
  if item.label_ in {"TIME","DATE"}:
    question("when",item)
  if item.label_ in {"PERSON","NORP"}:
    question("who",item)
  if item.label_ in {"FACILITY","FAC","GPE",'LOC',"GPE","GPE_LOC"}:
    question("where",item)
  if item.label_ in {"PRODUCT","EVENT","WORK_OF_ART","LAW","LANGUAGE","PROD","ORG","GPE_ORG"}:
    question("what",item)

IndentationError: ignored

In [4]:
##############################################_diff_##############################################

In [5]:

class cls_diff:
  
  def __init__(self, paragraph):
    self.paragraph = paragraph
    self.difference_synonyms_string = "contrast difference distinction contrary unlike unlike versus vs"
    self.sentences_list=[]
    self.list_of_questions=[]
    self.between_splitted_sentence=""
    self.prepare_seperating_and_synonyms_words()
    self.prepare_paragraph_to_0basewords_of_sentences_listoflist0()
    self.extract_sentences_that_contains_synonyms()
    self.basewords_of_sentences_listoflist
    
  def prepare_seperating_and_synonyms_words(self):
    #prepare difference_synonyms_basewords_list and seperator_words for matching (check_0difference_synonyms_basewords_list0_present_in_0basewords_of_sentences_listoflist0():)
    obj_preprocess = preprocess()
    self.difference_synonyms_basewords_list = obj_preprocess.convert_to_basewords(self.difference_synonyms_string)
    self.seperator_words = obj_preprocess.convert_to_basewords("and versus vs ,")
    
  def prepare_paragraph_to_0basewords_of_sentences_listoflist0(self):
    #text preperation into basewords for matching (check_0difference_synonyms_basewords_list0_present_in_0basewords_of_sentences_listoflist0():)
    text = self.paragraph
    preprocess_obj = preprocess()
    coref_text = preprocess_obj.replace_all_corefs(text)
    self.sentences_list=self.sentences_list+preprocess_obj.convert_para_to_listofsents(text)
    basewords_of_sentences_listoflist=[]
    for each_sentence in self.sentences_list:
      basewords_of_sentences_listoflist.append(preprocess_obj.convert_to_basewords(each_sentence))
    self.basewords_of_sentences_listoflist = basewords_of_sentences_listoflist

  def extract_sentences_that_contains_synonyms(self):
    #returns the index if the synonym of "difference" is present in any of sentences. 
    # variables: #synonyms of the word "difference" = difference_synonyms_basewords_list , sentences = basewords_of_sentences_listoflist
    basewords_of_sentences_listoflist = self.basewords_of_sentences_listoflist
    for index in range(len(basewords_of_sentences_listoflist)):
      for each_synonym in self.difference_synonyms_basewords_list:
        if each_synonym in basewords_of_sentences_listoflist[index]:
          #return index
          self.split_sentences_with_between_etc(index)  #each iteration main function
          break;
          #print("index=",index)
          
  def split_sentences_with_between_etc(self, index):
    if "between" in self.sentences_list[index].lower():
      betw_splitted_sentence=str(self.sentences_list[index].split("between")[1])
      self.betw_splitted_sentence=betw_splitted_sentence
      betw_splitted_sentence_words_listofstrngs=betw_splitted_sentence.split()
      self.split_sententences_with_and_etc(betw_splitted_sentence)
        #regex

  def split_sententences_with_and_etc(self, betw_splitted_sentence):
    #test_list = ['and', 'versus']
    test_list = self.seperator_words
    #betw_splitted_sentence = 'the main difference between power of z angle, x angle and y  angle is very interesting to teach'
    # code to split the line with 'diff' with keywords and and comma
    if re.search(r'\b({})\b'.format("|".join(test_list)),betw_splitted_sentence):
      splitted_parts_lstofstrngs = re.sub("[\s]and[\s]", "~",betw_splitted_sentence ) #change splittd parts lst of strings to before and after since this is a string
      splitted_parts_lstofstrngs = re.sub(",[\s]", "~", splitted_parts_lstofstrngs)
      splitted_parts_lstofstrngs = re.sub("[\s]versus[\s]", "~", splitted_parts_lstofstrngs)
      splitted_parts_lstofstrngs = re.sub("[\s]vs[\s]", "~", splitted_parts_lstofstrngs)
      splitted_parts_lstofstrngs = re.split("~",splitted_parts_lstofstrngs)
      self.extract_dependency_from_splitted_parts(splitted_parts_lstofstrngs)

  def extract_dependency_from_splitted_parts(self, splitted_parts_lstofstrngs):
      #code to extract dependencies from parts splitted by seperating words.
      words_and_deps_ofsplit_asdict={}
      dep_pattern_ofallsplits_asstrng=""
      words_and_deps_ofsplits_aslistofdict=[]
      for each_splitted_part in splitted_parts_lstofstrngs:
        for token in word_tokenize(each_splitted_part):
          words_and_deps_ofsplit_asdict[nltk.pos_tag([token])[0][0]] = nltk.pos_tag([token])[0][1] #0=>word, 1=>dep
          dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng+str("<")+nltk.pos_tag([token])[0][1]+str(">")
        words_and_deps_ofsplits_aslistofdict.append(words_and_deps_ofsplit_asdict)
        dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng+"~"
        words_and_deps_ofsplit_asdict={}
      dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng[:-1]
      dep_pattern_ofallsplits_aslstofstrngs=dep_pattern_ofallsplits_asstrng.split("~")
      self.find_smallest_split_pattern(dep_pattern_ofallsplits_aslstofstrngs)

  def find_smallest_split_pattern(self, dep_pattern_ofallsplits_aslstofstrngs):
    #choose smallest part of seperated line parts
    smallest_diff_line=dep_pattern_ofallsplits_aslstofstrngs[0]
    for i in dep_pattern_ofallsplits_aslstofstrngs[1:]:
      if len(i)<len(smallest_diff_line):
        smallest_diff_line=i
    largest_common_pattern_of_diff=self.create_all_ordered_compinations_of_word(smallest_diff_line)
    self.match_deppattern_in_sentence(largest_common_pattern_of_diff)

  def create_all_ordered_compinations_of_word (self, pattern1): 
    #code to get each ordered compination of a sentence to check for a pattern from in descending order of length
    for incr in range(len(pattern1)): #pattern1 is named so,  because renaming "pattern" might change other keywords with "pattern" in it
      decr=len(pattern1)-incr-1
      decr_cp=decr
      #print(line[0:k])
      sub_incr=0
      while decr_cp !=len(pattern1):
        return pattern1
        #use regex, if present, output that pattern.
        #print(pattern1[sub_incr:decr_cp+1]) #activate this line to print all ordered combinations of word in the terminal
        decr_cp=decr_cp+1
        sub_incr=sub_incr+1

  def match_deppattern_in_sentence(self,pattern1):
    token = word_tokenize(self.betw_splitted_sentence)
    tags = nltk.pos_tag(token)
    reg = "NN:{" + str(pattern1) + "}"
    a = nltk.RegexpParser(reg)
    result = a.parse(tags)
    list_of_contexts_ineachsentence=[]
    for subtree in result.subtrees():
      a=" ".join(w for w, t in subtree.leaves())
      list_of_contexts_ineachsentence.append( a ) 
    self.create_question_and_save(list_of_contexts_ineachsentence[1:])

  def create_question_and_save(self, list_of_contexts_ineachsentence_cpy):
    for i in range(len(list_of_contexts_ineachsentence_cpy)-1):
      list_of_contexts_ineachsentence_cpy[i] = list_of_contexts_ineachsentence_cpy[i] + str(",")

    list_of_contexts_ineachsentence_cpy[-1] = str("and ") + list_of_contexts_ineachsentence_cpy[-1]
    
    self.list_of_questions.append(str("What are the differences between " + " ".join(list_of_contexts_ineachsentence_cpy)))


In [6]:
test_obj = cls_diff( 'The main difference between a good book and a bad book is its difference. The main difference between a well written story and a poorly written story is its story.')
print(test_obj.list_of_questions)

['What are the differences between a good book, and a bad book', 'What are the differences between a well written story, and a poorly written story']


In [7]:
#notes:=
  #diff_list=["contrast","difference","distinction","contrary","different","differing","unlike","differ","unlike","versus","vs","v/s"]
  #diff_maybe_list=["the opposite","the contrary","the reverse","compare","on the other hand","while","whereas","but"]
  #seperator_words=["and",",","versus","vs"]
  #similarity_list=["resemblance similarity parallels analogy similar, analogous, common, comparable, identical, same, resemble, correspond, similarly, likewise, same, like, as"]

In [8]:
###################################################################_similarities_#######################################################

In [ ]:
pip install keybert


In [ ]:
################################################


# script.py
from keybert import KeyBERT

doc = """
    Supervised learning is the machine learning task of learning a function that
    maps an input to an output based on example input-output pairs. It infers a
    function from labeled training data consisting of a set of training examples.
    In supervised learning, each example is a pair consisting of an input object
    (typically a vector) and a desired output value (also called the supervisory signal). 
    A supervised learning algorithm analyzes the training data and produces an inferred function, 
    which can be used for mapping new examples. An optimal scenario will allow for the 
    algorithm to correctly determine the class labels for unseen instances. This requires 
    the learning algorithm to generalize from the training data to unseen situations in a 
    'reasonable' way (see inductive bias).
"""

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

print(kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None))
#[
#    ('learning', 0.4604),
#    ('algorithm', 0.4556),
#    ('training', 0.4487),
#    ('class', 0.4086),
#    ('mapping', 0.3700)
#]

[('supervised', 0.6676), ('labeled', 0.4896), ('learning', 0.4813), ('training', 0.4134), ('labels', 0.3947)]


where
['where', 'is', 'stronger', 'than', 'Pakistan', '.']
where
['where', 'is', 'stronger', 'than', 'where', '.']
where
['where', 'is', 'stronger', 'than', 'Afghanistan']
where
['where', 'is', 'stronger', 'than', 'where']


In [58]:
doc = nlp("India is my country")
for ents in doc.ents:
  print 

India


In [70]:
token = word_tokenize("India is my country")
print(token[0].text)

AttributeError: ignored

In [169]:
class l1:

  def __init__ (self, paragraph= "India is stronger than Pakistan. Pakistan is stronger than Afghanistan"):
    
    self.paragraph = paragraph
    self.coref_text = preprocess().replace_all_corefs(paragraph)
    self.doc = nlp1(self.coref_text)
    

    self.dep()
    self.ent_list_of_current_sentence = []
    self.iterate_sentences()

  def dep(self):
    subj=root_verb=obj =[]
    for token in self.doc:
      if token.dep_ in ["nsubjpass",'csubjpass','nsubj','csubj']:
        subj.append(token)
      if token.dep_ in ["ROOT","root"]:
        root_verb.append(token)
      if token.dep_ in ["dobj",'iobj','oprd','obj',"pobj"]:
        obj.append(token)
      return subj,root_verb, obj
      
  def refresh_entity_list(self, sentence):
    self.ent_list_of_current_sentence = []
    doc = nlp(sentence)
    for ent in doc.ents:
      self.ent_list_of_current_sentence.append(ent)

  def iterate_sentences(self):
   sentences_list = preprocess().convert_para_to_listofsents(self.coref_text)
   for sentence in sentences_list:
     self.refresh_entity_list(sentence)
     words_in_sentence = preprocess().convert_to_tokens(sentence)
     self.iter_words(words_in_sentence)

  def iter_words(self, words_in_sentence):
    for word in words_in_sentence:
      for ent in self.ent_list_of_current_sentence:
        if str(ent).lower() == word.lower():
          self.question(words_in_sentence) 
          return
    
  def question(self, words_in_sentence):
    for ent in self.ent_list_of_current_sentence:
      question = []
      for i in range(len(words_in_sentence)):
        if words_in_sentence[i].lower() == ent.text.lower():
          question = words_in_sentence.copy()
          wh_question_word = self.check_ent_label_type(ent.label_)
          
          print(question[i])

          question[i] = wh_question_word
          print("win sent",words_in_sentence)
          print(question)
          print("============")

           
  def check_ent_label_type(self, ent):
    if ent in {"TIME","DATE"}:
      return "when"
    elif ent in {"PERSON","NORP"}:
      return "who"
    elif ent in {"FACILITY","FAC","GPE",'LOC',"GPE","GPE_LOC"}:
      return "where"
    elif ent in {"PRODUCT","EVENT","WORK_OF_ART","LAW","LANGUAGE","PROD","ORG","GPE_ORG"}:
      return "what"





level1 = l1()

qst: ['India', 'is', 'stronger', 'than', 'Pakistan', '.']
qst word:  where
India
win sent ['India', 'is', 'stronger', 'than', 'Pakistan', '.']
['where', 'is', 'stronger', 'than', 'Pakistan', '.']
qst: ['India', 'is', 'stronger', 'than', 'Pakistan', '.']
qst word:  where
Pakistan
win sent ['India', 'is', 'stronger', 'than', 'Pakistan', '.']
['India', 'is', 'stronger', 'than', 'where', '.']
qst: ['Pakistan', 'is', 'stronger', 'than', 'Afghanistan']
qst word:  where
Pakistan
win sent ['Pakistan', 'is', 'stronger', 'than', 'Afghanistan']
['where', 'is', 'stronger', 'than', 'Afghanistan']
qst: ['Pakistan', 'is', 'stronger', 'than', 'Afghanistan']
qst word:  where
Afghanistan
win sent ['Pakistan', 'is', 'stronger', 'than', 'Afghanistan']
['Pakistan', 'is', 'stronger', 'than', 'where']
